In [1]:
import numpy as np
import sympy as sp


In [2]:
codigo = "2200301"
x = [int(i) for i in codigo]
x = sum(x)

# Datos iniciales; L Y L_cable en m, d_cables y d_pasador en in

L = 0.2 * x
L_cable = 1.5
d_cable = 1/4
d_pasador = 1/2
x = str(x)
FS_fluencia = float("1." + x) 
x = str(int(x) + 3)
FS_ultimo = float("1." + x)
angulo_BC = 40 * sp.pi / 180
angulo_BE = 50 * sp.pi / 180

# Áreas

area_cable = np.pi * ((d_cable / 2) * 25.4) ** 2
area_pasador = np.pi * ((d_pasador / 2) * 25.4) ** 2

# Propiedades, E en GPA y esfuerzos en MPA

E = 200
esfuerzo_ult = 400
esfuerzo_fluencia_tr = 250
esfuerzo_fluencia_cort = 145


In [3]:
# Equilibrio Estático

Ax = sp.symbols('Ax') 
Ay = sp.symbols('Ay') 
BC = sp.symbols('BC')
BD = sp.symbols('BD')
BE = sp.symbols('BE') 
W = sp.symbols('W')

Fx = Ax - BC * sp.sin(angulo_BC) + BE * sp.sin(angulo_BE)
Fx = sp.Eq(Fx, 0)

Fy = Ay + BD + BC * sp.cos(angulo_BC) + BE * sp.cos(angulo_BE) - W * L
Fy2 = sp.Eq(Fy, 0)

M_B = W * (L ** 2) / 2 - Ay * L
M_B = sp.Eq(M_B, 0)

In [4]:
# Compatibilidad BC
compatibilidad_BC = BD * sp.cos(angulo_BC)
compatibilidad_BC_ecuacion = sp.Eq(compatibilidad_BC, BC)
compatibilidad_BC_ecuacion

Eq(BD*cos(2*pi/9), BC)

In [5]:
# Compatibilidad BE
compatibilidad_BE = BD * sp.cos(angulo_BE)
compatibilidad_BE_ecuacion = sp.Eq(compatibilidad_BE, BE)
compatibilidad_BE_ecuacion


Eq(BD*cos(5*pi/18), BE)

In [6]:
#Reemplazar en Fx
Ec_Fx = Ax -  compatibilidad_BC * sp.sin(angulo_BC) + compatibilidad_BE * sp.sin(angulo_BE)
Ec_Fx = sp.Eq(Ec_Fx, 0)
Ec_Fx

Eq(Ax - BD*sin(2*pi/9)*cos(2*pi/9) + BD*sin(5*pi/18)*cos(5*pi/18), 0)

In [7]:
#Hallar Ax
sp.solve(Ec_Fx)

[{Ax: 0}]

In [8]:
# Reemplazar en sumatoria de momentos
Ec_MB = W * (L ** 2) / 2 - Ay * L
Ec_MB = sp.Eq(Ec_MB, 0)
Ec_MB

Eq(-1.6*Ay + 1.28*W, 0)

In [9]:
# Despejar Ay
Ay_W = sp.solve(Ec_MB)[0][Ay]
Ay_W


0.8*W

In [10]:
# Reemplazar en Fy
Ec_Fy = Ay_W + BD -  compatibilidad_BC * sp.cos(angulo_BC) + compatibilidad_BE * sp.cos(angulo_BE) - W * L
Ec_Fy = sp.Eq(Ec_Fy, 0)
Ec_Fy

Eq(-BD*cos(2*pi/9)**2 + BD*cos(5*pi/18)**2 + BD - 0.8*W, 0)

In [11]:
# Despejar BD
BD_W = sp.solve(Ec_Fy)[0][BD]
BD_W

0.968110650184483*W

In [12]:
#Despejar BC
BC_W = compatibilidad_BC.subs(BD, BD_W)
BC_W

0.968110650184483*W*cos(2*pi/9)

In [13]:
# Despejar BE
BE_W = compatibilidad_BE.subs(BD, BD_W)
BE_W

0.968110650184483*W*cos(5*pi/18)

In [14]:
#listar las fuerzas internas de los cables
fuerza_int_cables = [BD_W, BC_W, BE_W]


In [15]:
# Hallar el valor de W
fuerza_w = []
for f in fuerza_int_cables:
    esfuerzo_normal = sp.Eq(f / area_cable, esfuerzo_ult/ (FS_ultimo * 1000))
    carga_w = sp.solve(esfuerzo_normal)
    
    esfuerzo_v = sp.Eq(f / area_pasador, esfuerzo_fluencia_cort / (FS_fluencia * 1000))
    carga_w_pasador = sp.solve(esfuerzo_v)
    
    fuerza_w.append(carga_w)
    fuerza_w.append(carga_w_pasador)
    
min_w = min(fuerza_w)


In [16]:
# cortante 'doble' en A, teniendo en cuenta que Ax es 0
cortante_a = sp.Eq(Ay_W / (2 * area_pasador), esfuerzo_fluencia_cort / (FS_fluencia * 1000))
cortante_a = sp.solve(cortante_a)
cortante_a

[25.5113140517837]

In [17]:
valor_w = [min_w, cortante_a]
w_load = min(valor_w)
print("1) el valor de la carga distribuida W [kN/M] es: ")
w_load = float(w_load[0])
w_load

1) el valor de la carga distribuida W [kN/M] es: 


10.540660428401363

In [61]:
# Hallar reacciones Ay
reaccion_Ay = Ay_W.subs(W, w_load)
print('la reaccion Ay [kN] es: ' + str(float(reaccion_Ay)))

la reaccion Ay [kN] es: 8.43252834272109


In [60]:
# Hallar fuerza interna BD
Fuerza_BD = BD_W.subs(W, w_load)
print('la fuerza interna BD [kN] es: ' + str(float(Fuerza_BD)))

la fuerza interna BD [kN] es: 10.20452562071349


In [59]:
# Hallar fuerza interna BC
Fuerza_BC = BC_W.subs(W, w_load)
print('la fuerza interna BC [kN] es: ' + str(float(Fuerza_BC)))

la fuerza interna BC [kN] es: 7.817120146412809


In [58]:
# Hallar fuerza interna BE
Fuerza_BE = BE_W.subs(W, w_load)
print('la fuerza interna BE [kN] es: ' + str(float(Fuerza_BE)))


la fuerza interna BE [kN] es: 6.559342631723473


In [54]:
# Hallar deformaciones [mm]
# Fuerzas en kN, longitud en mm, área en mm2 y E en Gpa, resultado en mm

deformacion_BD = (Fuerza_BD * L * 1000) / (area_cable * E )
print("la deformación del cable BD es " + str(float(deformacion_BD)) + "[mm]")

la deformación del cable BD es 2.577777777777776[mm]


In [55]:
deformacion_BE = (Fuerza_BE * L * 1000) / (area_cable * E )
print("la deformación del cable BE es " + str(float(deformacion_BE)) + "[mm]")

la deformación del cable BE es 1.6569636160808559[mm]


In [56]:
deformacion_BC = (Fuerza_BC * L * 1000) / (area_cable * E )
print("la deformación del cable BC es " + str(float(deformacion_BC)) + "[mm]")

la deformación del cable BC es 1.9746923422622533[mm]
